In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Dependencia opcional para Fama-French
try:
    from pandas_datareader import data as pdr
    PDR_AVAILABLE = True
except Exception as e:
    PDR_AVAILABLE = False
    _pdr_err = e

# --- Parámetros ---
tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25])
inicio = "2025-09-08"
fin = "2025-09-24"
inversion_inicial = 1_000_000 * 18.5

# --- Descargar precios ---
# yfinance puede devolver un DataFrame con niveles ("Adj Close", ticker) o con columnas simples.
# Ser robustos: descargar y preferir "Adj Close", luego "Close", y manejar Series -> DataFrame.
data = yf.download(tickers, start=inicio, end=fin, progress=False, auto_adjust=False)

if isinstance(data, pd.DataFrame) and "Adj Close" in data:
    data = data["Adj Close"].copy()
elif isinstance(data, pd.DataFrame) and "Close" in data:
    data = data["Close"].copy()
else:
    # Si yfinance devolvió una Series (un solo ticker) o ya un DataFrame de precios
    data = data.copy()

# Asegurarnos de tener un DataFrame con columnas por ticker
if isinstance(data, pd.Series):
    data = data.to_frame()

# Normalizar nombres de columnas para que coincidan con la lista 'tickers' (insensible a mayúsculas)
def normalize_col(col):
    col_str = str(col)
    for t in tickers:
        if col_str.upper() == t.upper():
            return t
        if col_str.split(".")[0].upper() == t.split(".")[0].upper():
            return t
    return col

data.columns = [normalize_col(c) for c in data.columns]

# --- Rendimientos diarios ---
rend_diarios = data.pct_change().dropna()

# --- Rendimiento acumulado por activo ---
rend_acum = (1 + rend_diarios).prod() - 1

# --- Rendimiento portafolio ---
rend_port_diario = rend_diarios @ pesos
rend_port_total = (1 + rend_port_diario).prod() - 1

# --- Riesgo (desviación estándar) ---
riesgo_port = rend_port_diario.std() * np.sqrt(len(rend_port_diario))  # aprox anualizado corto

# --- Valor final ---
valor_final = inversion_inicial * (1 + rend_port_total)

print("\n--- Resultados ---")
print("Rendimiento acumulado por activo (%):")
print(rend_acum * 100)
print("\nRendimiento total portafolio: {:.2f}%".format(rend_port_total * 100))
print("Riesgo (desviación estándar aprox.): {:.2f}%".format(riesgo_port * 100))
print("Valor final del portafolio: ${:,.2f}".format(valor_final))

# --- Fama-French 3 Factores (diario) ---
# Descargamos factores de Fama-French desde pandas_datareader ('famafrench')
if not PDR_AVAILABLE:
    # Intento de instalación automática (opcional)
    import sys, subprocess
    try:
        print("\nInstalando pandas_datareader ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas_datareader", "-q"])
        from pandas_datareader import data as pdr
        PDR_AVAILABLE = True
        print("pandas_datareader instalado correctamente.")
    except Exception as e:
        print("No se pudo importar/instalar pandas_datareader. Instálalo manualmente con: pip install pandas_datareader")
        raise e

# Descargar factores diarios
ff = pdr.DataReader("F-F_Research_Data_Factors_Daily", "famafrench")
ff_df = ff[0].copy()

# Asegurar índice datetime y usar solo columnas necesarias
ff_df.index = pd.to_datetime(ff_df.index)
ff_df = ff_df[["Mkt-RF", "SMB", "HML", "RF"]].astype(float) / 100.0  # pasar de % a decimales

# Alinear con retornos del portafolio (usar el factor más reciente disponible antes de cada fecha del portafolio)
# Esto evita que la intersección de fechas quede vacía cuando los factores no llegan hasta las mismas fechas.
rp = rend_port_diario.copy().rename("Rp")

# Preparar dataframes para merge_asof (ambos ordenados por fecha)
rp_df = rp.reset_index().rename(columns={"index": "Date"})
ff_reset = ff_df.reset_index().rename(columns={"index": "Date"}).sort_values("Date")
rp_df = rp_df.sort_values("Date")

# Para cada fecha de rp, tomar los factores de la fecha más reciente <= esa fecha
df_ff3 = pd.merge_asof(rp_df, ff_reset, on="Date", direction="backward")

# Si aún hay NaNs (por ejemplo si no existen factores previos a alguna fecha), intentar intersección clásica
if df_ff3[["Mkt-RF", "SMB", "HML", "RF"]].isnull().any(axis=None):
    df_ff3 = pd.concat([rp, ff_df], axis=1, join="inner").dropna()
    # volver a indexar por fecha
    if not df_ff3.empty:
        df_ff3.index = pd.to_datetime(df_ff3.index)

# Si seguimos sin datos alineados, informar y saltar la regresión
if df_ff3.empty:
    print("\nNo hay fechas solapadas entre los retornos del portafolio y los factores Fama-French. La regresión no puede ejecutarse.")
else:
    # Asegurar índice datetime si usamos merge_asof
    if "Date" in df_ff3.columns:
        df_ff3.set_index("Date", inplace=True)

    # Exceso de retorno del portafolio
    df_ff3["Rp_excess"] = df_ff3["Rp"] - df_ff3["RF"]

    # Regresión OLS: Rp - RF = alpha + beta_m*(Mkt-RF) + s*SMB + h*HML + error
    X = df_ff3[["Mkt-RF", "SMB", "HML"]]
    # Forzar adición de constante de forma explícita (esto crea la columna 'const')
    X = sm.add_constant(X, has_constant="add")
    y = df_ff3["Rp_excess"]

    # Comprobar que X e y no están vacíos
    if X.shape[0] == 0 or y.shape[0] == 0:
        print("\nDatos insuficientes para estimar el modelo (X o y vacíos).")
    else:
        modelo_ff3 = sm.OLS(y, X).fit()

        print("\n--- Modelo Fama-French 3 Factores (con datos diarios) ---")
        print(modelo_ff3.summary())

        # Obtener alfa de forma segura (evitar KeyError si el nombre del intercepto difiere o no existe)
        if "const" in modelo_ff3.params.index:
            alpha_diaria = modelo_ff3.params["const"]
        elif "Intercept" in modelo_ff3.params.index:
            alpha_diaria = modelo_ff3.params["Intercept"]
        else:
            # Si no hay intercepto en el modelo, aproximamos alfa como la media de los residuales
            alpha_diaria = float(modelo_ff3.resid.mean())

        alpha_anual = (1 + alpha_diaria) ** 252 - 1
        print("\nAlfa diaria: {:.4f}%".format(alpha_diaria * 100))
        print("Alfa anualizada (aprox): {:.2f}%".format(alpha_anual * 100))

        # Extraer betas de forma robusta (si faltara alguna, quedará como NaN)
        betas = modelo_ff3.params.reindex(["Mkt-RF", "SMB", "HML"])
        print("\nBetas estimadas (Mkt-RF, SMB, HML):")
        print(betas)


--- Resultados ---
Rendimiento acumulado por activo (%):
GCARSOA1.MX      4.408903
GFINBURO.MX      2.788692
HOTEL.MX         2.339179
PINFRA.MX        4.082304
TLEVISACPO.MX   -3.710755
dtype: float64

Rendimiento total portafolio: 1.67%
Riesgo (desviación estándar aprox.): 2.83%
Valor final del portafolio: $18,809,871.55


C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_21352\3014552003.py:90: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader("F-F_Research_Data_Factors_Daily", "famafrench")



--- Modelo Fama-French 3 Factores (con datos diarios) ---
                            OLS Regression Results                            
Dep. Variable:              Rp_excess   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 29 Sep 2025   Prob (F-statistic):                nan
Time:                        19:05:39   Log-Likelihood:                 33.487
No. Observations:                  10   AIC:                            -64.97
Df Residuals:                       9   BIC:                            -64.67
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

c:\Users\Juan Marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [4]:
data

,GCARSOA1.MX,GFINBURO.MX,HOTEL.MX,PINFRA.MX,TLEVISACPO.MX
Date,,,,,
2025-09-08,128.149994,50.919998,3.42,243.000000,10.51
2025-09-09,126.379997,51.049999,3.44,243.889999,10.37
2025-09-10,123.029999,50.500000,3.48,245.759995,10.21
2025-09-11,127.739998,51.740002,3.50,250.399994,10.15
2025-09-12,127.019997,52.299999,3.50,251.559998,10.45
2025-09-15,130.699997,53.220001,3.50,250.160004,10.37
2025-09-17,131.809998,52.090000,3.48,247.979996,10.25
2025-09-18,132.000000,52.529999,3.50,249.940002,9.98
2025-09-19,136.279999,50.799999,3.50,245.080002,9.89
